In [2]:
cd ..

/Users/olumide/Documents/Self Improvement - ML/MLapps/ML_EndtoEnd_Projects/sleep_state_detector


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

In [4]:
train_series = pd.read_parquet("child-mind-institute-detect-sleep-states/train_series.parquet")
train_events = pd.read_csv("child-mind-institute-detect-sleep-states/train_events.csv")

In [ ]:
# #Splitting by person - For a first try 70:25:5
# split_1 = int(0.7 * len(train_series.series_id.unique()))
# split_2 = int(0.95 * len(train_series.series_id.unique()))

# train, dev, test = train_series.series_id.unique()[:split_1], train_series.series_id.unique()[split_1:split_2], train_series.series_id.unique()[split_2:]
# train_set, dev_set, test_set = train_series[train_series['series_id'].isin(train)],train_series[train_series['series_id'].isin(dev)], train_series[train_series['series_id'].isin(test)]
# test_set

In [4]:
def split_data_by_series(df, col_name):
    first_split = int(0.7 * len(df[col_name].unique()))
    second_split = int(0.95 * len(df[col_name].unique()))
    train_seriesid_list, dev_seriesid_list, test_seriesid_list = df[col_name].unique()[:first_split], df[col_name].unique()[first_split:second_split], df[col_name].unique()[second_split:]
    trainset, devset, testset = df[df[col_name].isin(train_seriesid_list)], df[df[col_name].isin(dev_seriesid_list)], df[df[col_name].isin(test_seriesid_list)]
    return trainset, devset, testset

train_set, dev_set, test_set = split_data_by_series(df=train_series, col_name='series_id')
train_event, dev_event, test_event = split_data_by_series(df=train_events, col_name='series_id')

In [5]:
def convert_to_datetime(df, col_name):
    df[col_name] = pd.to_datetime(df[col_name], utc=True)
    return df

train_set = convert_to_datetime(train_set, col_name='timestamp')
train_event = convert_to_datetime(train_event, col_name='timestamp')

/var/folders/1t/7mfrmh2s0r17tk3yntzzmsjw0000gn/T/ipykernel_35575/311105558.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = pd.to_datetime(df[col_name], utc=True)
/var/folders/1t/7mfrmh2s0r17tk3yntzzmsjw0000gn/T/ipykernel_35575/311105558.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = pd.to_datetime(df[col_name], utc=True)


##  From Literature there are 2 broad ways to engineer the timesries data:

 - Temporal embedding
 - Time delay embedding

### Time delay embedding
We assume an arbitary window length that is less than the lenght of the time series. We then slide the window along the time series and extract the data points within the window. This is repeated until the end of the time series is reached. The window length is a hyperparameter that needs to be tuned. The different time delay embedding techniques are explained below

- Lags/backshifts: If we are time t, we can take 2 previous values and combine it with the current value at time t to form a vector of length 3. This is called a lag of 2. We can also take 3 previous values and combine it with the current value at time t to form a vector of length 4. This is called a lag of 3. We can continue this process to form a lag of n. Lags connect the point to n number of previous points.

- Rolling window aggregation: In this method we are connecting the present point to an aggregate statistic  of a window from the past. The window can be of any length. 

- Seasonal rolling window aggregation:  Similar to rolling window but instead of taking n consectutive previous ppoint and computing an aggregate statistic, this takes a seasonal window and computes the statistic , skipping a constsnt number of timesteps. For example, if we have a seasonal window of 4 and a skip of 2, then we will take the first 4 points and compute the aggregate statistic. Then we will skip 2 points and take the next 4 points and compute the aggregate statistic. This process is repeated until the end of the time series is reached. 

- Exponential weighted aggregation: In this method we are connecting the present point to an aggregate statistic  of a window from the past. The window can be of any length. The difference between this and rolling window is that the weights of the points in the window are not equal. The weights are exponentially decreasing as we go back in time. The most recent point has the highest weight and the weight decreases exponentially as we go back in time.

In [ ]:
#Lag/backshift in feature engineering -  This method will have an issue with jsut the first few observations 

test = train_set[0:5000]
test["anglez_lag_of_3"] = test['anglez'].shift(2) #taking into consideration the previous 2 point together with the current point hence why shift = 2
test.head()

In [ ]:
test.tail()

In [ ]:
## Rolling window aggregation - This method 

test['angle_rollingmean_3'] =  test['anglez'].shift(1).rolling(12).mean()
test.head()

## Feature Engineering

In [ ]:
# # Starting with the rolling window system -  Mean, median, std for both features
# train_set['rolling_anglez_mean'] = train_set['anglez'].shift(1).rolling(12).mean().fillna(method='bfill')
# train_set['rolling_anglez_median'] = train_set['anglez'].shift(1).rolling(12).median().fillna(method='bfill')
# train_set['rolling_anglez_std'] = train_set['anglez'].shift(1).rolling(12).std().fillna(method='bfill')
# train_set['rolling_enmo_mean'] = train_set['enmo'].shift(1).rolling(12).mean().fillna(method='bfill')
# train_set['rolling_enmo_median'] = train_set['enmo'].shift(1).rolling(12).median().fillna(method='bfill')
# train_set['rolling_enmo_std'] = train_set['enmo'].shift(1).rolling(12).std().fillna(method='bfill')

# train_set

In [6]:
def engineer_feature(df):
    df['rolling_anglez_mean'] = df['anglez'].shift(1).rolling(12).mean().fillna(method='bfill')
    df['rolling_anglez_median'] = df['anglez'].shift(1).rolling(12).median().fillna(method='bfill')
    df['rolling_anglez_std'] = df['anglez'].shift(1).rolling(12).std().fillna(method='bfill')
    df['rolling_enmo_mean'] = df['enmo'].shift(1).rolling(12).mean().fillna(method='bfill')
    df['rolling_enmo_median'] = df['enmo'].shift(1).rolling(12).median().fillna(method='bfill')
    df['rolling_enmo_std'] = df['enmo'].shift(1).rolling(12).std().fillna(method='bfill')
    return df

train_set = engineer_feature(df=train_set)
dev_set = engineer_feature(df=dev_set)

/var/folders/1t/7mfrmh2s0r17tk3yntzzmsjw0000gn/T/ipykernel_35575/2574174594.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['rolling_anglez_mean'] = df['anglez'].shift(1).rolling(12).mean().fillna(method='bfill')
/var/folders/1t/7mfrmh2s0r17tk3yntzzmsjw0000gn/T/ipykernel_35575/2574174594.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rolling_anglez_mean'] = df['anglez'].shift(1).rolling(12).mean().fillna(method='bfill')
/var/folders/1t/7mfrmh2s0r17tk3yntzzmsjw0000gn/T/ipykernel_35575/2574174594.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['rollin

In [7]:
def clean_eventdf(df, col_name):
    df = df.drop(df[df[col_name].isna()].index).reset_index()
    return df

train_event_cleaned = clean_eventdf(df=train_event, col_name='step')
dev_event_cleaned = clean_eventdf(df=dev_event, col_name='step')

### Assigning Labels to the training data

In [8]:
merged = train_set.merge(train_event_cleaned, on=['series_id', 'step'], how='left')
merged

,series_id,step,timestamp_x,anglez,enmo,rolling_anglez_mean,rolling_anglez_median,rolling_anglez_std,rolling_enmo_mean,rolling_enmo_median,rolling_enmo_std,index,night,event,timestamp_y
0,038441c925bb,0,2018-08-14 19:30:00+00:00,2.6367,0.0217,2.689800,2.63680,0.143648,0.021775,0.02170,0.000427,NaN,NaN,NaN,NaT
1,038441c925bb,1,2018-08-14 19:30:05+00:00,2.6368,0.0215,2.689800,2.63680,0.143648,0.021775,0.02170,0.000427,NaN,NaN,NaN,NaT
2,038441c925bb,2,2018-08-14 19:30:10+00:00,2.6370,0.0216,2.689800,2.63680,0.143648,0.021775,0.02170,0.000427,NaN,NaN,NaN,NaT
3,038441c925bb,3,2018-08-14 19:30:15+00:00,2.6368,0.0213,2.689800,2.63680,0.143648,0.021775,0.02170,0.000427,NaN,NaN,NaN,NaT
4,038441c925bb,4,2018-08-14 19:30:20+00:00,2.6368,0.0215,2.689800,2.63680,0.143648,0.021775,0.02170,0.000427,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87376315,bf00506437aa,671215,2019-07-09 13:44:35+00:00,-12.6228,0.0796,-13.886592,-14.07400,2.842582,0.078383,0.07230,0.024777,NaN,NaN,NaN,NaT
87376316,bf00506437aa,671216,2019-07-09 13:44:40+00:00,-11.0369,0.0347,-13.543758,-12.81200,2.712699,0.080250,0.07480,0.023863,NaN,NaN,NaN,NaT
87376317,bf00506437aa,671217,2019-07-09 13:44:45+00:00,-15.2801,0.0389,-12.998308,-12.67280,2.474422,0.076725,0.07230,0.027268,NaN,NaN,NaN,NaT
87376318,bf00506437aa,671218,2019-07-09 13:44:50+00:00,-10.4895,0.0503,-12.838658,-12.67280,2.228561,0.073967,0.07185,0.029382,NaN,NaN,NaN,NaT


In [ ]:
#Quick sanity check
test = merged[:100_000]
test[test.event.notnull()]

In [ ]:
# #Optimized approach to creating label for the sleep windows

# merged['label'] = 'awake'
# onset_indices = merged[merged.event == 'onset'].index
# wakeup_indices = merged[merged.event == 'wakeup'].index
# mask = np.zeros(len(merged), dtype=bool)
# for onset_idx, wakeup_idx in zip(onset_indices, wakeup_indices):
#     mask[onset_idx:wakeup_idx] = True
# merged.loc[mask, 'label'] = "sleep"
# merged

In [12]:
def assign_label(df, col_name):
    df['label'] = 'awake'
    sleep_indices = df[df[col_name]== 'onset'].index
    awake_indices = df[df[col_name] == 'wakeup'].index
    empty_mask_arr = np.zeros(len(df), dtype=bool)
    for sleep_idx, awake_idx in zip(sleep_indices, awake_indices):
        empty_mask_arr[sleep_idx:awake_idx] = True
    df.loc[empty_mask_arr, 'label'] = 'sleep'
    return df
    
merged = assign_label(df=merged, col_name='event')

In [13]:
merged[91_294:91_300]

,series_id,step,timestamp_x,anglez,enmo,rolling_anglez_mean,rolling_anglez_median,rolling_anglez_std,rolling_enmo_mean,rolling_enmo_median,rolling_enmo_std,index,night,event,timestamp_y,label
91294,038441c925bb,91294,2018-08-20 02:17:50+00:00,-66.515900,0.0119,-65.700485,-65.895050,0.493225,0.008375,0.00815,0.000614,NaN,NaN,NaN,NaT,awake
91295,038441c925bb,91295,2018-08-20 02:17:55+00:00,-71.569397,0.0128,-65.876026,-65.947300,0.344331,0.008592,0.00815,0.001174,NaN,NaN,NaN,NaT,awake
91296,038441c925bb,91296,2018-08-20 02:18:00+00:00,-71.095100,0.0094,-66.420792,-65.969501,1.636109,0.008892,0.00815,0.001690,10.0,6.0,onset,2018-08-20 02:18:00+00:00,sleep
91297,038441c925bb,91297,2018-08-20 02:18:05+00:00,-70.937302,0.0094,-66.877401,-65.979702,2.092077,0.008967,0.00815,0.001691,NaN,NaN,NaN,NaT,sleep
91298,038441c925bb,91298,2018-08-20 02:18:10+00:00,-70.886299,0.0092,-67.303184,-66.003201,2.361635,0.009017,0.00815,0.001694,NaN,NaN,NaN,NaT,sleep
91299,038441c925bb,91299,2018-08-20 02:18:15+00:00,-70.917503,0.0090,-67.722142,-66.042850,2.522565,0.009025,0.00815,0.001695,NaN,NaN,NaN,NaT,sleep


In [14]:
merged[97848:97865]

,series_id,step,timestamp_x,anglez,enmo,rolling_anglez_mean,rolling_anglez_median,rolling_anglez_std,rolling_enmo_mean,rolling_enmo_median,rolling_enmo_std,index,night,event,timestamp_y,label
97848,038441c925bb,97848,2018-08-20 11:24:00+00:00,-36.129902,0.0069,-31.746092,-32.033650,3.079170,0.008358,0.00795,0.002591,NaN,NaN,NaN,NaT,sleep
97849,038441c925bb,97849,2018-08-20 11:24:05+00:00,-36.270699,0.0067,-32.686125,-32.206751,2.437305,0.008383,0.00795,0.002574,NaN,NaN,NaN,NaT,sleep
97850,038441c925bb,97850,2018-08-20 11:24:10+00:00,-36.123699,0.0065,-33.459942,-32.530001,1.870943,0.007617,0.00740,0.001052,NaN,NaN,NaN,NaT,sleep
97851,038441c925bb,97851,2018-08-20 11:24:15+00:00,-36.597801,0.0064,-33.846308,-33.131849,1.904961,0.007467,0.00715,0.001074,NaN,NaN,NaN,NaT,sleep
97852,038441c925bb,97852,2018-08-20 11:24:20+00:00,-37.141701,0.0065,-34.204208,-34.457649,1.990191,0.007367,0.00700,0.001115,NaN,NaN,NaN,NaT,sleep
97853,038441c925bb,97853,2018-08-20 11:24:25+00:00,-39.715900,0.0077,-34.636283,-35.571449,2.020513,0.007308,0.00680,0.001143,NaN,NaN,NaN,NaT,sleep
97854,038441c925bb,97854,2018-08-20 11:24:30+00:00,-42.092201,0.0078,-35.283633,-35.928999,2.305182,0.007250,0.00680,0.001099,NaN,NaN,NaN,NaT,sleep
97855,038441c925bb,97855,2018-08-20 11:24:35+00:00,-44.051998,0.0089,-36.132417,-36.126801,2.775884,0.007308,0.00680,0.001109,NaN,NaN,NaN,NaT,sleep
97856,038441c925bb,97856,2018-08-20 11:24:40+00:00,-45.718899,0.0094,-37.127542,-36.200300,3.294921,0.007350,0.00680,0.001162,NaN,NaN,NaN,NaT,sleep
97857,038441c925bb,97857,2018-08-20 11:24:45+00:00,-45.859402,0.0089,-38.207700,-36.434250,3.815405,0.007350,0.00680,0.001162,NaN,NaN,NaN,NaT,sleep


## Preliminary Modelling

In [ ]:
## Random Forest for peliminary timeseries modelling
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, max_depth=12, min_samples_leaf=40)
